In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments

c:\Users\Denise\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
import torch
import torch.nn as nn
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [ ]:
model_name = "gklmip/bert-tagalog-base-uncased"
BERT_MODEL = AutoModel.from_pretrained(model_name, return_dict=True)
BERT_TOKENIZER = AutoTokenizer.from_pretrained(model_name)

Some weights of BertModel were not initialized from the model checkpoint at gklmip/bert-tagalog-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_path = './dataset/train-try.csv'
val_path = './dataset/val-try.csv'
test_path = './dataset/test-try.csv'
dataset_path = './dataset/mlthsc.csv'

In [ ]:
df = pd.read_csv(dataset_path)
train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)
LABELS = ['Age', 'Gender', 'Physical', 'Race', 'Religion', 'Others']

# Investigating the flow of a single input to BERT output

Kuha tayo ng isang sample text mula sa dataset natin (index 10 sa mlthsc.csv)

tas kunin yung representation as input to BERT

In [ ]:
data_row = df.iloc[10]

hate_speech_text = data_row['Text']
labels = data_row[LABELS]


encoding = BERT_TOKENIZER.encode_plus(
    hate_speech_text,
    add_special_tokens=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=128,
    return_token_type_ids=False,
    return_attention_mask=True
)


representation = {
    'input_ids': encoding['input_ids'].flatten(),
    'attention_mask': encoding['attention_mask'].flatten(),
    'labels': torch.FloatTensor(labels)
}


# Sample input


101 = [CLS]
18215 = ng
2229 = mga
102 = [SEP]
0 = [PAD]
...


In [ ]:
print(hate_speech_text)
encoding 

Tangina ng mga girl gamer sa ml ambobobo nyo mag ros nalang kayo


{'input_ids': tensor([[  101, 18215,  2229,  1745,  1762,  3780,  3629,  1122,  1741, 13305,
         33900,  2573,  2573,  5193,  1814,  3438,  8288,  2921,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

# Output ng BERT

In [ ]:
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

output = BERT_MODEL(input_ids, attention_mask=attention_mask)
output




BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.8359,  0.4488,  0.9444,  ..., -0.7361, -0.0301, -0.4596],
         [-0.2004, -1.6600, -0.0554,  ..., -0.2445,  1.3696,  0.6472],
         [-0.0274, -1.0908, -0.5952,  ..., -0.5364,  0.5854, -0.1231],
         ...,
         [ 0.1735, -0.3333, -0.1038,  ...,  0.2210,  0.5366,  0.2195],
         [ 0.6714, -0.5266, -0.3117,  ...,  0.5718, -0.7893,  0.4809],
         [ 0.4980, -0.0049,  0.6744,  ...,  0.1516, -0.6812,  0.7967]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.2852, -0.0088, -0.4217,  0.5029, -0.5775,  0.8261, -0.3018,  0.3172,
          0.8902, -0.3816,  0.6675, -0.4433, -0.1287,  0.0369, -0.3644,  0.2761,
         -0.3168, -0.7029, -0.5461, -0.6946, -0.8801, -0.3294, -0.4730, -0.1471,
         -0.3901,  0.6406,  0.4061, -0.5049,  0.3023, -0.0564,  0.3540, -0.0063,
         -0.3668,  0.5559,  0.0557,  0.4991,  0.1714, -0.5220,  0.4974,  0.1893,
         -0.6577,  0.4095, -0.27

## [CLS] Token

*last_hidden_state* nagrerepresent ng embedding ng CLS token, which is a tensor of shape (768) 

In [ ]:
cls_embedding = output.last_hidden_state[0, 0, :]
cls_embedding

tensor([ 8.3589e-01,  4.4881e-01,  9.4441e-01,  7.4970e-01,  1.0030e+00,
        -1.9352e+00,  3.1232e-01, -1.1254e-01,  3.8781e-01,  4.1114e-01,
        -9.6400e-01,  9.4284e-01,  2.3553e-01,  1.3223e+00,  4.3053e-01,
        -6.5761e-01,  3.9198e-01,  7.7849e-02,  2.6433e-01,  8.9570e-01,
        -1.0710e+00, -2.1475e+00,  7.6227e-01,  6.0151e-01,  7.9761e+00,
        -2.6235e-01, -1.7995e+00,  8.6282e-01,  8.0839e-01, -1.9218e-01,
         2.2226e-02,  2.0101e+00, -1.3445e+00, -9.7304e-02,  7.2417e-01,
        -2.2077e-01,  5.6884e-01,  3.3936e-01, -1.1537e+00,  3.0945e-01,
        -8.8095e-01,  8.8641e-01, -4.0872e-01,  8.1410e-01,  2.4943e-01,
        -1.5284e+00, -1.5121e-01,  7.4530e-01, -1.2743e-01,  8.8579e-01,
        -8.6979e-01,  7.8161e-01, -3.3494e-01, -2.1311e+00, -5.4304e-01,
        -6.4087e-01, -1.7144e-01,  6.6150e-02,  8.4312e-01,  2.2235e-01,
         3.7095e-01, -6.9918e-01,  1.6027e-01, -1.3652e+00,  3.7835e-01,
        -8.8689e-01,  6.7411e-01,  1.1421e-01, -6.6

## Classification Layer (Linear/Dense/Fully Connected)

on top of BERT
untrained pa eto kaya random pa ang weights

-> ang output ay *logits* (raw score)

In [ ]:
classifier = nn.Linear(768, 6)  # 6 labels
logits = classifier(cls_embedding)
logits


tensor([-0.3199, -0.4695, -0.0829,  0.2421,  0.0986, -0.7168],
       grad_fn=<AddBackward0>)

# Sigmoid Activation
para maconvert ang logits as a number between 0 and 1

In [ ]:
probabilities = torch.sigmoid(logits)
print(probabilities)

tensor([0.4207, 0.3847, 0.4793, 0.5602, 0.5246, 0.3281],
       grad_fn=<SigmoidBackward0>)


# Apply threshold = 0.5

Probabilities greater than 0.5 means the label exists and below means label does not exist

In [ ]:
threshold = 0.5
y_pred = np.zeros(probabilities.shape)
y_pred[np.where(probabilities >= threshold)] = 1

y_pred

array([0., 0., 0., 1., 1., 0.])

In [ ]:
predicted_labels = [label for label, pred in zip(LABELS, y_pred) if pred == 1]
print("Predicted labels:", predicted_labels)

Predicted labels: ['Race', 'Religion']


# Hamming Loss per instance

Fraction of wrong labels to all labels

**Formula:**
```
          number of incorrect labels
    HL = -----------------------------
           total number of labels
```

In [ ]:
# get true labels
labels = representation['labels'] # [0, 1, 0, 0, 0, 0]
y_true = np.zeros(labels.shape)
y_true[np.where(labels >= threshold)] = 1
print(y_true)
true_labels = [label for label, true_label in zip(LABELS, y_true) if true_label == 1]
print("True labels:", true_labels)


[0. 1. 0. 0. 0. 0.]
True labels: ['Gender']


In [ ]:
print(y_true)
print(y_pred)

xor_result = np.logical_xor(y_true, y_pred)

print(xor_result, "(True means label is misclassified, False if correct label)")
xor_sum = np.sum(xor_result)
print(xor_sum, "wrong labels" )

total_labels = len(y_true)
hamming_loss = xor_sum / total_labels
print("Hamming Loss:", hamming_loss)



[0. 1. 0. 0. 0. 0.]
[0. 0. 0. 1. 1. 0.]
[False  True False  True  True False] (True means label is misclassified, False if correct label)
3 wrong labels
Hamming Loss: 0.5
